<a href="https://colab.research.google.com/github/annhienktuit/UIT-AI-Challenge2020/blob/master/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Xây dựng bộ phát hiện đối tượng với thuật toán YOLOv3 được viết bằng thư viện Keras. Thư viện này bao gồm đầy đủ các module: phát hiện đối tượng với pre-trainded model, huấn luyện lại mô hình, phát hiện đối tượng ảnh trên ảnh/video/webcam.

Đầu tiên, ta sẽ khởi tạo Google Colab với sử dụng 1 GPU. Sau đó thực hiện kết nối với Google Drive của tài khoản cá nhân như sau:

In [ ]:
!nvidia-smi

Fri Jul 31 04:13:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
lab_path = "/content/gdrive/My\ Drive/AdvancedComputerVision/keras-yolo3"

Sau đó, sử dụng git để clone thư mục project từ github của tác giả experiencor với đường dẫn như sau. Lưu ý rằng, ta nên đặt thực project kế bên file jupyter notebook này để tiện thao tác.

In [ ]:
!git clone https://github.com/experiencor/keras-yolo3.git $lab_path

Sau đó tiến hành cài đặt các thư viện cần thiết để có thể sử dụng được keras-yolo3. Đây là bước thực hiện quan trọng vì thông thường các thư viện sẽ sử dụng các thư viện bên thứ 3 với các phiên bản cố định. Do đó, để đảm bảo thư viện keras-yolo3 sử dụng trơn tru thì ta sẽ cài đặt các thư viện kèm theo trong file requirements.txt.

In [ ]:
!pip install -r $lab_path/requirements.txt

## 1. Thử nghiệm với pretrained model của tác giả thuật toán YOLO.

File mô hình huấn luyện sẵn được đặt ở đường dẫn sau https://pjreddie.com/media/files/yolov3.weights
Lưu ý là file mô hình cần được đặt trong thư mục gốc của thư viện để tiện cho việc thực hiện.

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -O $lab_path/yolov3.weights

Sau đó, ta tiến hành sử dụng pretrained model để phát hiện đối tượng trong ảnh cho trước. Sử dụng chương trình yolo3_one_file_to_detect_them_all.py với file mô hình đã download ở file trước đó để phát hiện các đối tượng trong ảnh. Tập dữ liệu các đối tượng phát hiện được lấy từ tập PASCAL VOC 2012.

Ảnh kết quả phát hiện đối tượng trên pretrained model cho tập dữ liệu này được minh hoạ như ở hình sau:

![alt text](https://i.imgur.com/6nun5fl.jpg)

In [ ]:
!python $lab_path/yolo3_one_file_to_detect_them_all.py -w $lab_path/yolov3.weights -i $lab_path/dog-tales-img.jpg

#2. Huấn luyện dữ liệu với keras-yolo3

![alt text](https://i.imgur.com/45SRniL.jpg)

Bước tiếp theo, ta sẽ tiến hành huấn luyện trên tập dữ liệu mới là gấu mèo (raccoon). Tập dữ liệu này được tải từ https://github.com/experiencor/raccoon_dataset.git

Lưu ý rằng, ta sẽ chỉ sử dụng 2 thư mục của github này là images và annotations. Trong đó,

*   images: thư mục chứa các ảnh về gấu mèo
*   annotations: tương ứng với từng file ảnh của thư mục images, ta có một file .xml chứa thông tin về vị trí chính xác của đối tượng (gấu mèo) trong ảnh.

Các bạn nên xem kỹ định dạng của các file XML này để biết cấu trúc file gán nhãn phục vụ cho việc thực hiện đồ án sau này.



In [ ]:
!git clone https://github.com/experiencor/raccoon_dataset.git $lab_path/raccoon_dataset

Một file khác cũng rất quan trọng của các mô hình huấn luyện học sâu nói chung và YOLO nói riêng là file cấu hình (config) các tham số khi huấn luyện.

Trong thư viện này, file config được đặt trong thư mục /zoo. Ví dụ như config_raccoon.json. Các tham số cơ bản của file config như:
- Đường dẫn đến thư mục train, validation (mặc định chia tỉ lệ train:validation = 8:2 nếu không có tham số gắn sẵn)
- GPU sử dụng để huấn luyện
- File pretrained cho tập image net backend.h5
- Learning rate
- Batch size

Sau khi đã cấu hình file config, ta sẽ tiến hành huấn luyện:

Để chương trình train có thể chạy được ta phải tải thêm file backend.h5 chứa pretrained model của backbone model và đặt ở thư mục lab_path:
https://drive.google.com/file/d/1f3Ylajkfg69iXc5159oJlYsq-9D_Px50/view?usp=sharing

In [ ]:
!python $lab_path/train.py -c $lab_path/zoo/config_raccoon.json

Sau khi quá trình huấn luyện kết thúc, hệ thống sẽ lưu mô hình raccoon.h5 trong thư mục của máy ảo colab. Ta có thể sửa lại mã nguồn của thư viện để các file mô hình và file trung gian đặt trong thư mục của thư viện cho gọn gàng.

Sau đó, ta sẽ tiến hành thử nghiệm mô hình đã huấn luyện trên ảnh mẫu lấy từ mạng internet hoặc thư mục ảnh validation/train.

In [ ]:
!python $lab_path/predict.py -c $lab_path/zoo/config_raccoon.json -i $lab_path/raccoon_dataset/images/raccoon-200.jpg

#3. Train trên bộ dữ liệu giao thông

Như vậy, trong phần bài tập này chúng ta đã làm quen với thư viện keras-yolo3. Trong phần đồ án giữa kỳ, ta sẽ tiến hành trên dữ liệu video giao thông được thu tại TP. Hồ Chí Minh. Ví dụ, ảnh sau được cắt tại góc đường Nam Kỳ Khởi Nghĩa - Võ Thị Sáu.

![alt text](https://i.imgur.com/NQBeNpG.jpg)

Mục tiêu của đồ án này là sử dụng thuật toán YOLOv3 để tiến hành ước lượng lưu lượng giao thông theo các hướng đi.

Đầu tiên, ta sẽ thử sử dụng pretrained model lên dữ liệu video thực tế.

Sau đó tiến hành xây dựng bộ dữ liệu gán nhãn cho ngữ cảnh camera giao thông.

Huấn luyện lại mô hình với dữ liệu gán nhãn mới.

Phát hiện và đếm các đối tượng xe máy, xe hơi trong video giao thông.

Video test: https://drive.google.com/file/d/1u__2tl79nSevF_pnvwsVd79Hu1aWLJQE/view?usp=sharing

Tập ảnh dùng để train: https://drive.google.com/file/d/1-ZT6V2OrIWIWtLFFlizQBVvYxL__QFL7/view?usp=sharing

